In [1]:
import cugraph
import cudf
from cugraph.datasets import email_Eu_core

el = email_Eu_core.get_edgelist(download=True)

G = cugraph.Graph(directed=True)
G.from_cudf_edgelist(el, source="src", destination="dst")

seeds = G.select_random_vertices(62, int(0.0001 * len(el)))

In [6]:
sampling_results_renumbered, offsets_renumbered, renumber_map = cugraph.uniform_neighbor_sample(
    G,
    seeds,
    [5,5],
    with_replacement=False,
    with_edge_properties=True,
    with_batch_ids=False,
    deduplicate_sources=True,
    exclude_sources=True,
    renumber=True,
    return_offsets=True,
    random_state=62,
    use_legacy_names=False,
    compress_per_hop=True,
    compression='COO',
    include_hop_column=False,
)

/home/nfs/abarghi/cugraph6/python/cugraph/cugraph/sampling/uniform_neighbor_sample.py:244: FutureWarning: The with_edge_properties flag is deprecated and will be removed in the next release.
  warnings.warn(warning_msg, FutureWarning)


In [7]:
sampling_results_renumbered

,majors,minors,weight,edge_id,edge_type
0,0,0,<NA>,<NA>,<NA>
1,0,1,<NA>,<NA>,<NA>
2,0,2,<NA>,<NA>,<NA>
3,0,3,<NA>,<NA>,<NA>
4,0,4,<NA>,<NA>,<NA>
5,1,0,<NA>,<NA>,<NA>
6,0,1,<NA>,<NA>,<NA>
7,2,1,<NA>,<NA>,<NA>
8,4,1,<NA>,<NA>,<NA>
9,0,3,<NA>,<NA>,<NA>


In [16]:
offsets_renumbered

,offsets,batch_id
0,0,0
1,5,<NA>
2,27,<NA>


In [17]:
renumber_map

,map,renumber_map_offsets
0,<NA>,0
1,<NA>,22
